In [ ]:
import json
import tensorflow as tf

label_file = open("./imagenet-sample/labels.json")
label_file = json.load(label_file)
labels_to_name = label_file['classes']
img_to_label = label_file['labels']
img_names = list(img_to_label.keys())

json_items = json.load(open("./imagenet-sample/imagenet_class_index.json")).items()
imageNetClasses = {labels[1]: int(idx) for (idx, labels) in json_items}
def get_class_idx(label):
	return imageNetClasses.get(label, None)

In [ ]:
#To select GPU
device = "/device:GPU:1"
gpus = tf.config.list_physical_devices('GPU')

tf.config.set_visible_devices(gpus[4], 'GPU')

# To check the Model accuracy

In [ ]:
# import necessary packages
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import decode_predictions
from tensorflow.keras.applications.resnet50 import preprocess_input
import tensorflow as tf
import numpy as np
import argparse
import cv2

#Check Accuracy of model on Original Images and Perturbed Images
org_acc = 0
adv_acc = 0

def preprocess_image(image):
	# swap color channels, resize the input image
	image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
	image = cv2.resize(image, (224, 224))
	# return the preprocessed image
	return image

model = ResNet50(weights="imagenet")

img_list = []
true_label_list = []

for img in img_names:
    image = cv2.imread('./imagenet-sample/data/'+img+'.jpg')
    image = preprocess_image(image)
    image = tf.constant(image, dtype=tf.float32)
    img_list.append(image)
    true_label_list.append(img_to_label[img])

img_list = np.array(img_list)

preprocessedImage = preprocess_input(tf.constant(img_list, dtype=tf.float32))

predictions = model.predict(preprocessedImage)
predictions = np.array(decode_predictions(predictions, top=3))[:,0,1]

for index1 in range(0,1000):
    if(true_label_list[index1] == get_class_idx(predictions[index1])):
        org_acc+=1

perturbations = np.load('perturbations.npy').squeeze()
scale = 1

preprocessedImage = preprocess_input(tf.constant(tf.clip_by_value((img_list+scale*perturbations), 0, 255).numpy().astype('uint8'), dtype=tf.float32))

predictions = model.predict(preprocessedImage)
predictions = np.array(decode_predictions(predictions, top=3))[:,0,1]

for index1 in range(0,1000):
    if(true_label_list[index1] == get_class_idx(predictions[index1])):
        adv_acc+=1

print(org_acc) #716
print(adv_acc) # 58_1, 57_2, 56_4, 57_8, 33_16

# To check trasferability

In [ ]:
# import necessary packages
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import decode_predictions
from tensorflow.keras.applications.resnet50 import preprocess_input
import tensorflow as tf
import numpy as np
import argparse
import cv2

def preprocess_image(image):
	# swap color channels, resize the input image
	image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
	image = cv2.resize(image, (224, 224))
	# return the preprocessed image
	return image

i = 0

model = ResNet50(weights="imagenet")

img_list = []
true_label_list = []

for img in img_names:
    image = cv2.imread('./imagenet-sample/data/'+img+'.jpg')
    image = preprocess_image(image)
    adversarialImage = tf.constant(image, dtype=tf.float32)
    img_list.append(adversarialImage)
    true_label_list.append(img_to_label[img])

img_list = np.array(img_list)

acc_list = []
perturbations = np.load('perturbations.npy')
scale = 1

for index in range(0,1000):
    predictions = model.predict(preprocess_input(tf.constant(tf.clip_by_value((img_list+scale*perturbations[index]), 0, 255).numpy().astype('uint8'), dtype=tf.float32)))
    predictions = np.array(decode_predictions(predictions, top=3))[:,0,1]
    
    count = 0
    accuracy = 0
    for index1 in range(0,1000):
        if(index == index1):
            continue
        count+=1
        if(true_label_list[index1] == get_class_idx(predictions[index1])):
            accuracy+=1
    acc_list.append(accuracy)
    print(str(index)+' '+str(count))

In [ ]:
print(np.array(acc_list).sum()/9990.0) # 712858,71.36_1 -- 711005,71.17_2 -- 697854,69.85_4 -- 704216,70.49_8 -- 693092,69.38_16

# print(org_acc) #716
# print(adv_acc) #60(jpg), 57(png)

# Plot the images with their label, predicted label and cofidence of prediction

In [ ]:
#Plot the images with their label, predicted label and cofidence of prediction
import matplotlib.pyplot as plt

ind_list = [0, 100, 200, 300, 400, 500, 600, 700, 800, 900]

for ind in ind_list:
    img = img_names[ind]

    image = cv2.imread('./imagenet-sample/data/'+img+'.jpg')
    testImage = cv2.resize(cv2.cvtColor(image, cv2.COLOR_BGR2RGB), (224, 224))
    image = preprocess_image(image)
    baseImage = tf.constant(image, dtype=tf.float32)
    
    image = cv2.imread('./adv_png/'+img+'_adversarial.png')
    testImage_adv = cv2.resize(cv2.cvtColor(image, cv2.COLOR_BGR2RGB), (224, 224))
    image = preprocess_image(image)
    adversarialImage = tf.constant(image, dtype=tf.float32)
    
    preprocessedImage = preprocess_input(baseImage)
    predictions = model.predict(preprocessedImage)
    predictions = decode_predictions(predictions, top=3)[0]
    label = labels_to_name[get_class_idx(predictions[0][1])]
    confidence = predictions[0][2] * 100
    
    preprocessedImage = preprocess_input(adversarialImage)
    predictions = model.predict(preprocessedImage)
    predictions = decode_predictions(predictions, top=3)[0]
    label_adv = labels_to_name[get_class_idx(predictions[0][1])]
    confidence_adv = predictions[0][2] * 100
        
    fig, ax = plt.subplots(1, 2, figsize = (12, 6))
    fig.suptitle(labels_to_name[img_to_label[img]],size=30, fontweight='bold')
    fig.tight_layout(pad = 0.5, rect = (0,0.08,1,1))
    
    ax[0].imshow(testImage)
    ax[0].axis("off")
    ax[0].set_title(f"Prediction : {label} \n Confidence : {confidence:.3f}%", size = 20, y=-0.15)
    ax[1].imshow(testImage_adv)
    ax[1].axis("off")
    ax[1].set_title(f"Prediction : {label_adv} \n Confidence : {confidence_adv:.3f}%", size = 20, y=-0.15)
    plt.savefig('op_png/'+str(ind)+'.jpg')

In [ ]:
import matplotlib.pyplot as plt

ind_list = [[0, 100], [200, 300], [400, 500], [600, 700], [800, 900]]

fig, ax = plt.subplots(2, 5, figsize = (80, 16))
fig.tight_layout(pad = 0.0)

for ind in range(5):
    image1 = cv2.cvtColor(cv2.imread('op_png/'+str(ind_list[ind][0])+'.jpg'), cv2.COLOR_BGR2RGB)
    image2 = cv2.cvtColor(cv2.imread('op_png/'+str(ind_list[ind][1])+'.jpg'), cv2.COLOR_BGR2RGB)
    ax[0, ind].imshow(image1)
    ax[0, ind].axis("off")
    ax[1, ind].imshow(image2)
    ax[1, ind].axis("off")
plt.savefig('output_image.jpg')

# Effect of Noise

In [ ]:
#Effect of noise

# import necessary packages
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import decode_predictions
from tensorflow.keras.applications.resnet50 import preprocess_input
import tensorflow as tf
import numpy as np
import argparse
import cv2

import numpy as np

def preprocess_image(image):
	# swap color channels, resize the input image
	image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
	image = cv2.resize(image, (224, 224))
	# return the preprocessed image
	return image

model = ResNet50(weights="imagenet")

img_list = []
true_label_list = []

for img in img_names:
    image = cv2.imread('./imagenet-sample/data/'+img+'.jpg')
    image = preprocess_image(image)
    adversarialImage = tf.constant(image, dtype=tf.float32)
    img_list.append(adversarialImage)
    true_label_list.append(img_to_label[img])

img_list = np.array(img_list)

acc_list = []
noise_list = [1, 2, 4, 8, 16, 32, 64, 128]

for noise in noise_list:
    np.random.seed(0)
    
    predictions = model.predict(preprocess_input(tf.constant(tf.clip_by_value((img_list+np.random.normal(size = img_list.shape, loc = 0, scale = noise)), 0, 255).numpy().astype('uint8'), dtype=tf.float32)))
    predictions = np.array(decode_predictions(predictions, top=3))[:,0,1]
    
    accuracy = 0
    for index1 in range(0,1000):
        if(true_label_list[index1] == get_class_idx(predictions[index1])):
            accuracy+=1
    acc_list.append(accuracy)

In [ ]:
print(acc_list) #(714, 714, 704, 693, 611, 463, 139, 3)

## Plot Accuracy of Random Noise compared to Adversarial Noise

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize = (12, 6))

x_noise = [0, 1, 2, 3, 4, 5, 6, 7]
y_noise = [71.4, 71.4, 70.4, 69.3, 61.1, 46.3, 13.9, 0.3]
x_adv = [0, 1, 2, 3, 4]
y_adv = [5.8, 5.7, 5.6, 5.7, 3.3]

x_orig = [-1, 9]
y_orig = [71.6, 71.6]

plt.xlim([0, 8])
plt.ylim([0, 100])

plt.xlabel('EPS/Std-Dev (log scale)', size = 'xx-large')
plt.ylabel('Accuracy (%)', size = 'xx-large')

plt.plot(x_orig, y_orig, label = 'Base Accuracy', ls = '--')
plt.plot(x_adv, y_adv, label = 'Accuracy on Adversarial Images')
plt.plot(x_noise, y_noise, label = 'Accuracy on Random Gaussian Perturbation')

plt.xticks(range(0, 9), [1, 2, 4, 8, 16, 32, 64, 128, 256])

plt.legend()
plt.grid()

plt.savefig('noise_vs_adv.jpg')

# Transferability to Different Models

In [ ]:
#Transferability to Different Models

# import necessary packages
import tensorflow as tf
import numpy as np
import argparse
import cv2

import numpy as np

import tensorflow.keras.applications.mobilenet_v2 as mn

def preprocess_image(image):
	# swap color channels, resize the input image
	image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
	image = cv2.resize(image, (224, 224))
	# return the preprocessed image
	return image

img_list = []
true_label_list = []

for img in img_names:
    image = cv2.imread('./imagenet-sample/data/'+img+'.jpg')
    image = preprocess_image(image)
    image = tf.constant(image, dtype=tf.float32)
    img_list.append(image)
    true_label_list.append(img_to_label[img])

img_list = np.array(img_list)

model = mn.MobileNetV2(weights="imagenet")

acc = 0

predictions = model.predict(mn.preprocess_input(tf.constant(img_list, dtype=tf.float32)))
predictions = np.array(mn.decode_predictions(predictions, top=3))[:,0,1]

for index1 in range(0,1000):
    if(true_label_list[index1] == get_class_idx(predictions[index1])):
        acc+=1
print(acc) #709

acc = 0
perturbations = np.load('perturbations.npy').squeeze()
scale = 1

predictions = model.predict(mn.preprocess_input(tf.constant(tf.clip_by_value((img_list+scale*perturbations), 0, 255).numpy().astype('uint8'), dtype=tf.float32)))
predictions = np.array(mn.decode_predictions(predictions, top=3))[:,0,1]

for index1 in range(0,1000):
    if(true_label_list[index1] == get_class_idx(predictions[index1])):
        acc+=1
print(acc) # 658_1, 601_2, 490_4, 536_8, 486_16, _32, _64 / 331_4*2, 198_8*2

In [ ]:
#Transferability to Different Models

# import necessary packages
import tensorflow as tf
import numpy as np
import argparse
import cv2

import numpy as np

import tensorflow.keras.applications.vgg16 as vgg

def preprocess_image(image):
	# swap color channels, resize the input image
	image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
	image = cv2.resize(image, (224, 224))
	# return the preprocessed image
	return image

img_list = []
true_label_list = []

for img in img_names:
    image = cv2.imread('./imagenet-sample/data/'+img+'.jpg')
    image = preprocess_image(image)
    image = tf.constant(image, dtype=tf.float32)
    img_list.append(image)
    true_label_list.append(img_to_label[img])

img_list = np.array(img_list)

model =  vgg.VGG16(weights="imagenet")

acc = 0

predictions = model.predict(vgg.preprocess_input(tf.constant(img_list, dtype=tf.float32)))
predictions = np.array(vgg.decode_predictions(predictions, top=3))[:,0,1]

for index1 in range(0,1000):
    if(true_label_list[index1] == get_class_idx(predictions[index1])):
        acc+=1
print(acc) #651

acc = 0
perturbations = np.load('perturbations.npy').squeeze()
scale = 1

predictions = model.predict(vgg.preprocess_input(tf.constant(tf.clip_by_value((img_list+scale*perturbations), 0, 255).numpy().astype('uint8'), dtype=tf.float32)))
predictions = np.array(vgg.decode_predictions(predictions, top=3))[:,0,1]

for index1 in range(0,1000):
    if(true_label_list[index1] == get_class_idx(predictions[index1])):
        acc+=1
print(acc) # 604_1, 569_2, 475_4, 528_8, 480_16 / 323_4*2, 189_8*2

# To Save the Images otained after applying perturbation

In [ ]:
#To save the images

def preprocess_image(image):
	# swap color channels, resize the input image, and add a batch
	# dimension
	image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
	image = cv2.resize(image, (224, 224))
	image = np.expand_dims(image, axis=0)
	# return the preprocessed image
	return image

perturbation_list = np.load('perturbations_float.npy')

for i in range(0,1000):
	print(i+1)
	img = img_names[i]
	image = cv2.imread('./imagenet-sample/data/'+img+'.jpg')
	image = preprocess_image(image)
	baseImage = tf.constant(image, dtype=tf.float32)
	adverImage = (baseImage + perturbation_list[i]).numpy().squeeze()
	adverImage = np.clip(adverImage, 0, 255).astype("uint8")
	adverImage = cv2.cvtColor(adverImage, cv2.COLOR_RGB2BGR)
	cv2.imwrite('./adv_png/'+img+'_adversarial.png', adverImage)